In [1]:
# Courtesy: https://github.com/mchablani/deep-learning/blob/master/sentiment-rnn/Sentiment_RNN.ipynb
# Depression Analysis in Bangla with LSTM-RNN::
# Two classes with balanced dataset (Total 1968 tweets):
# 1. Depressive (984 tweets)
# 2. Non-depressive (984 tweets)
# 1 depressive -> 1 non-depressive -> 1 depressive -> 1 non-depressive ...

'''
SELECT tweets_depressive_nondepressive_balanced_rearranged.tweet
INTO OUTFILE "D:/EDUCATION/Machine & Deep Learning/Implementations/tweets_depressive_nondepressive_balanced_rearranged_text.txt" 
FROM tweets_depressive_nondepressive_balanced_rearranged;
'''

'\nSELECT tweets_depressive_nondepressive_balanced_rearranged.tweet\nINTO OUTFILE "D:/EDUCATION/Machine & Deep Learning/Implementations/tweets_depressive_nondepressive_balanced_rearranged_text.txt" \nFROM tweets_depressive_nondepressive_balanced_rearranged;\n'

In [2]:
import numpy as np
import tensorflow as tf
from timeit import default_timer as timer
from collections import Counter
from string import punctuation

In [3]:
with open('tweets_depressive_nondepressive_balanced_rearranged_text.txt', 'r', encoding="utf8") as f:
    tweets = f.read()
with open('tweets_depressive_nondepressive_balanced_rearranged_labels.txt', 'r', encoding="utf8") as f:
    labels_org = f.read()

In [4]:
# Data preprocessing::
all_text = ''.join([c for c in tweets if c not in punctuation])
tweets = all_text.split('\n')

all_text = ' '.join(tweets)
words = all_text.split()

In [5]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

tweets_ints = []
for each in tweets:
    tweets_ints.append([vocab_to_int[word] for word in each.split()])

In [6]:
# Encoding the labels::
list_labels = []

for l in labels_org.split():
    if l == "non_depressive":
        list_labels.append(0)
    elif l == "depressive":
        list_labels.append(1)
        
labels = np.array(list_labels)
#print(len(labels))

In [7]:
tweets_lens = Counter([len(x) for x in tweets_ints])
print("Zero-length tweets: {}".format(tweets_lens[0]))
print("Maximum tweets length: {}".format(max(tweets_lens)))

Zero-length tweets: 1
Maximum tweets length: 61


In [8]:
# Filter out that tweets with 0 length
tweets_ints = [r[0:200] for r in tweets_ints if len(r) > 0]

In [9]:
from collections import Counter
tweets_lens = Counter([len(x) for x in tweets_ints])
print("Zero-length tweets: {}".format(tweets_lens[0]))
print("Maximum tweet length: {}".format(max(tweets_lens)))

Zero-length tweets: 0
Maximum tweet length: 61


In [10]:
seq_len = 200
features = np.zeros((len(tweets_ints), seq_len), dtype=int)
# print(features[:10,:100])
for i, row in enumerate(tweets_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
#features[:10,:100]

In [11]:
# 80% training, 10% validation, 10% testing::

split_frac = 0.8

split_index = int(split_frac * len(features))

train_x, val_x = features[:split_index], features[split_index:] 
train_y, val_y = labels[:split_index], labels[split_index:]

split_frac = 0.5
split_index = int(split_frac * len(val_x))

val_x, test_x = val_x[:split_index], val_x[split_index:]
val_y, test_y = val_y[:split_index], val_y[split_index:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nValidation label set: \t{}".format(val_y.shape),
      "\nTest label set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(1574, 200) 
Validation set: 	(197, 200) 
Test set: 		(197, 200)
label set: 		(1574,) 
Validation label set: 	(197,) 
Test label set: 		(197,)


In [12]:
# Build the graph::

lstm_size = 128
lstm_layers = 10
batch_size = 1
learning_rate = 0.001

In [13]:
n_words = len(vocab_to_int) + 1 # Add 1 for 0 added to vocab

# Create the graph object
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, None], name="inputs")
    labels_ = tf.placeholder(tf.int32, [None, None], name="labels")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")

In [14]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [15]:
def lstm_cell():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    # Add dropout to the cell
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

with tf.name_scope("RNN_layers"):
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [16]:
with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)

In [17]:
# Output::

with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    tf.summary.histogram('predictions', predictions)
with tf.name_scope('cost'):
    cost = tf.losses.mean_squared_error(labels_, predictions)
    tf.summary.scalar('cost', cost)

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

merged = tf.summary.merge_all()

In [18]:
# Validation accuracy::

with tf.name_scope('validation'):
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
# Batching::

def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [20]:
# Training::

epochs = 10
saver = tf.train.Saver()
start = timer()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/tb/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/tb/test', sess.graph)
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            summary, loss, state, _ = sess.run([merged, cost, final_state, optimizer], feed_dict=feed)
#             loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)

            train_writer.add_summary(summary, iteration)
        
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
#                     batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
            test_writer.add_summary(summary, iteration)
            saver.save(sess, "checkpoints/tweet_depressive_nondepressive_balanced_2.ckpt")
#            tensorboard = TensorBoard(log_dir="logs/tweet_5000_all_sentiments_six_classes-{}".format(int(time.time())))
    saver.save(sess, "checkpoints/tweet_depressive_nondepressive_balanced_2.ckpt")
    
duration = timer() - start
print('Time elasped =',duration,'sec(s)')

Epoch: 0/10 Iteration: 5 Train loss: 0.257
Epoch: 0/10 Iteration: 10 Train loss: 0.275
Epoch: 0/10 Iteration: 15 Train loss: 0.238
Epoch: 0/10 Iteration: 20 Train loss: 0.295
Epoch: 0/10 Iteration: 25 Train loss: 0.196
Val acc: 0.503
Epoch: 0/10 Iteration: 30 Train loss: 0.184
Epoch: 0/10 Iteration: 35 Train loss: 0.138
Epoch: 0/10 Iteration: 40 Train loss: 0.236
Epoch: 0/10 Iteration: 45 Train loss: 0.764
Epoch: 0/10 Iteration: 50 Train loss: 0.129
Val acc: 0.497
Epoch: 0/10 Iteration: 55 Train loss: 0.179
Epoch: 0/10 Iteration: 60 Train loss: 0.281
Epoch: 0/10 Iteration: 65 Train loss: 0.257
Epoch: 0/10 Iteration: 70 Train loss: 0.611
Epoch: 0/10 Iteration: 75 Train loss: 0.212
Val acc: 0.503
Epoch: 0/10 Iteration: 80 Train loss: 0.509
Epoch: 0/10 Iteration: 85 Train loss: 0.439
Epoch: 0/10 Iteration: 90 Train loss: 0.459
Epoch: 0/10 Iteration: 95 Train loss: 0.276
Epoch: 0/10 Iteration: 100 Train loss: 0.080
Val acc: 0.497
Epoch: 0/10 Iteration: 105 Train loss: 0.308
Epoch: 0/10 Ite

Epoch: 0/10 Iteration: 865 Train loss: 0.418
Epoch: 0/10 Iteration: 870 Train loss: 0.318
Epoch: 0/10 Iteration: 875 Train loss: 0.131
Val acc: 0.503
Epoch: 0/10 Iteration: 880 Train loss: 0.334
Epoch: 0/10 Iteration: 885 Train loss: 0.277
Epoch: 0/10 Iteration: 890 Train loss: 0.230
Epoch: 0/10 Iteration: 895 Train loss: 0.287
Epoch: 0/10 Iteration: 900 Train loss: 0.309
Val acc: 0.497
Epoch: 0/10 Iteration: 905 Train loss: 0.252
Epoch: 0/10 Iteration: 910 Train loss: 0.122
Epoch: 0/10 Iteration: 915 Train loss: 0.216
Epoch: 0/10 Iteration: 920 Train loss: 0.450
Epoch: 0/10 Iteration: 925 Train loss: 0.257
Val acc: 0.497
Epoch: 0/10 Iteration: 930 Train loss: 0.400
Epoch: 0/10 Iteration: 935 Train loss: 0.256
Epoch: 0/10 Iteration: 940 Train loss: 0.307
Epoch: 0/10 Iteration: 945 Train loss: 0.344
Epoch: 0/10 Iteration: 950 Train loss: 0.255
Val acc: 0.503
Epoch: 0/10 Iteration: 955 Train loss: 0.181
Epoch: 0/10 Iteration: 960 Train loss: 0.268
Epoch: 0/10 Iteration: 965 Train loss: 0

Val acc: 0.503
Epoch: 1/10 Iteration: 1705 Train loss: 0.235
Epoch: 1/10 Iteration: 1710 Train loss: 0.198
Epoch: 1/10 Iteration: 1715 Train loss: 0.457
Epoch: 1/10 Iteration: 1720 Train loss: 0.188
Epoch: 1/10 Iteration: 1725 Train loss: 0.189
Val acc: 0.503
Epoch: 1/10 Iteration: 1730 Train loss: 0.308
Epoch: 1/10 Iteration: 1735 Train loss: 0.366
Epoch: 1/10 Iteration: 1740 Train loss: 0.335
Epoch: 1/10 Iteration: 1745 Train loss: 0.426
Epoch: 1/10 Iteration: 1750 Train loss: 0.165
Val acc: 0.497
Epoch: 1/10 Iteration: 1755 Train loss: 0.184
Epoch: 1/10 Iteration: 1760 Train loss: 0.160
Epoch: 1/10 Iteration: 1765 Train loss: 0.395
Epoch: 1/10 Iteration: 1770 Train loss: 0.123
Epoch: 1/10 Iteration: 1775 Train loss: 0.161
Val acc: 0.503
Epoch: 1/10 Iteration: 1780 Train loss: 0.460
Epoch: 1/10 Iteration: 1785 Train loss: 0.267
Epoch: 1/10 Iteration: 1790 Train loss: 0.111
Epoch: 1/10 Iteration: 1795 Train loss: 0.229
Epoch: 1/10 Iteration: 1800 Train loss: 0.338
Val acc: 0.503
Epoch

Epoch: 1/10 Iteration: 2545 Train loss: 0.212
Epoch: 1/10 Iteration: 2550 Train loss: 0.312
Val acc: 0.503
Epoch: 1/10 Iteration: 2555 Train loss: 0.316
Epoch: 1/10 Iteration: 2560 Train loss: 0.292
Epoch: 1/10 Iteration: 2565 Train loss: 0.231
Epoch: 1/10 Iteration: 2570 Train loss: 0.350
Epoch: 1/10 Iteration: 2575 Train loss: 0.116
Val acc: 0.503
Epoch: 1/10 Iteration: 2580 Train loss: 0.244
Epoch: 1/10 Iteration: 2585 Train loss: 0.226
Epoch: 1/10 Iteration: 2590 Train loss: 0.180
Epoch: 1/10 Iteration: 2595 Train loss: 0.135
Epoch: 1/10 Iteration: 2600 Train loss: 0.207
Val acc: 0.503
Epoch: 1/10 Iteration: 2605 Train loss: 0.226
Epoch: 1/10 Iteration: 2610 Train loss: 0.160
Epoch: 1/10 Iteration: 2615 Train loss: 0.312
Epoch: 1/10 Iteration: 2620 Train loss: 0.113
Epoch: 1/10 Iteration: 2625 Train loss: 0.352
Val acc: 0.503
Epoch: 1/10 Iteration: 2630 Train loss: 0.304
Epoch: 1/10 Iteration: 2635 Train loss: 0.318
Epoch: 1/10 Iteration: 2640 Train loss: 0.191
Epoch: 1/10 Iteratio

KeyboardInterrupt: 

In [21]:
# Testing::

test_acc = []
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/tweet_depressive_nondepressive_balanced_2.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/tweet_depressive_nondepressive_balanced_2.ckpt
Test accuracy: 0.503
